Exemplo 1 - Gravado num dataset

In [ ]:
import h5py
import numpy as np
# ---------------------------
# 1) Definição dos tipos
# ---------------------------
# Strings de tamanho fixo
t_rua    = h5py.string_dtype(encoding='ascii', length=64)
t_cidade = h5py.string_dtype(encoding='ascii', length=32)
t_cep    = h5py.string_dtype(encoding='ascii', length=16)
t_nome   = h5py.string_dtype(encoding='ascii', length=64)

# Tipo composto Endereco
t_endereco = np.dtype([
    ("rua",    t_rua),
    ("cidade", t_cidade),
    ("cep",    t_cep),
])

# Tipo composto Pessoa com array fixo de 3 endereços
t_pessoa = np.dtype([
    ("nome", t_nome),
    ("idade", np.int32),
    ("enderecos", t_endereco, (3,)),  # shape (3,) de t_endereco
])

# ---------------------------
# 2) Dados de exemplo
# ---------------------------
dados = np.zeros(2, dtype=t_pessoa)

dados[0]["nome"] = "Marcos Tavares"
dados[0]["idade"] = 55
dados[0]["enderecos"][0] = ("Rua A, 123", "Rio de Janeiro", "20000-000")
dados[0]["enderecos"][1] = ("Av. Brasil, 500", "Niteroi", "24000-000")
dados[0]["enderecos"][2] = ("Rua das Flores, 10", "Petropolis", "25600-000")

dados[1]["nome"] = "Bruno Lima"
dados[1]["idade"] = 41
dados[1]["enderecos"][0] = ("Rua X, 1", "Sao Paulo", "01000-000")
dados[1]["enderecos"][1] = ("Av. Paulista, 1000", "Sao Paulo", "01310-100")
dados[1]["enderecos"][2] = ("Rua Y, 99", "Campinas", "13000-000")

# ---------------------------
# 3) Criação do arquivo HDF5 e dataset
# ---------------------------
file_path = "pessoas.h5"
with h5py.File(file_path, "w") as f:
    dset = f.create_dataset("pessoas", data=dados, dtype=t_pessoa)

print(f"Arquivo HDF5 criado: {file_path}")

Exemplo - Cadastro de pessoas com matriz de salario no ano

In [ ]:
import h5py
import numpy as np

# ---------------------------
# 1) Definição dos tipos
# ---------------------------
t_nome   = h5py.string_dtype(encoding='ascii', length=64)
t_rua    = h5py.string_dtype(encoding='ascii', length=64)
t_cidade = h5py.string_dtype(encoding='ascii', length=32)
t_cep    = h5py.string_dtype(encoding='ascii', length=16)

# Tipo composto Endereco
t_endereco = np.dtype([
    ("rua",    t_rua),
    ("cidade", t_cidade),
    ("cep",    t_cep),
])

# Tipo composto Pessoa
t_pessoa = np.dtype([
    ("nome", t_nome),
    ("idade", np.int32),
    ("enderecos", t_endereco, (3,)),  # 3 endereços
])

# ---------------------------
# 2) Criação dos dados
# ---------------------------
dados_pessoas = np.zeros(2, dtype=t_pessoa)

# Pessoa 0
dados_pessoas[0]["nome"] = "Ana Souza"
dados_pessoas[0]["idade"] = 34
dados_pessoas[0]["enderecos"][0] = ("Rua A, 123", "Rio de Janeiro", "20000-000")
dados_pessoas[0]["enderecos"][1] = ("Av. Brasil, 500", "Niteroi", "24000-000")
dados_pessoas[0]["enderecos"][2] = ("Rua das Flores, 10", "Petropolis", "25600-000")

# Pessoa 1
dados_pessoas[1]["nome"] = "Bruno Lima"
dados_pessoas[1]["idade"] = 41
dados_pessoas[1]["enderecos"][0] = ("Rua X, 1", "Sao Paulo", "01000-000")
dados_pessoas[1]["enderecos"][1] = ("Av. Paulista, 1000", "Sao Paulo", "01310-100")
dados_pessoas[1]["enderecos"][2] = ("Rua Y, 99", "Campinas", "13000-000")

# Matriz de salários 12x2 (12 meses, 2 pessoas)
salarios = np.array([
    [5000, 7000],
    [5050, 7020],
    [5100, 7050],
    [5200, 7100],
    [5250, 7200],
    [5300, 7250],
    [5400, 7300],
    [5450, 7350],
    [5500, 7400],
    [5550, 7500],
    [5600, 7600],
    [5700, 7700],
], dtype=np.float64)

# ---------------------------
# 3) Criação do arquivo HDF5
# ---------------------------
file_path = "cadastro_pessoas_salarios.h5"

with h5py.File(file_path, "w") as f:
    # Cria grupo "Cadastro"
    grp = f.create_group("Cadastro")

    # Cria datasets
    ds_pessoas = grp.create_dataset("pessoas", data=dados_pessoas, dtype=t_pessoa)
    ds_salarios = grp.create_dataset("salarios", data=salarios, dtype=np.float64)

    # Adiciona atributo
    ds_pessoas.attrs["numero"] = 2
    ds_salarios.attrs["meses"] = 12
    ds_salarios.attrs["tipo"] = "mensal"



print(f"Arquivo criado com sucesso: {file_path}")


Exemplo 3 - Leitura de dados e calculo de medias e leitura de atributos

In [ ]:
import h5py
import numpy as np

# Caminho do arquivo gerado anteriormente
file_path = "cadastro_pessoas_salarios.h5"

# Abre o arquivo em modo leitura
with h5py.File(file_path, "r") as f:
    # Acessa o dataset de salários dentro do grupo "Cadastro"
    salarios = f["Cadastro/salarios"][:]

    # Mostra o atributo meses (opcional)
    meses = f["Cadastro/salarios"].attrs.get("meses", None)
    print(f"Atributo 'meses': {meses}")

    # Calcula média geral (todas pessoas, todos meses)
    media_geral = np.mean(salarios)

    # Média por pessoa (colunas)
    media_por_pessoa = np.mean(salarios, axis=0)

print("Matriz de salários (12x2):")
print(salarios)
print("\nMédia geral de salários:", round(media_geral, 2))
print("Média por pessoa:", np.round(media_por_pessoa, 2))


Exemplo 4 - Integrando com o Pandas

In [1]:
import h5py
import pandas as pd

# Caminho do arquivo
file_path = "cadastro_pessoas_salarios.h5"

# Abre o arquivo HDF5
with h5py.File(file_path, "r") as f:
    # Lê o dataset "salarios" dentro do grupo "Cadastro"
    salarios = f["Cadastro/salarios"][:]
    meses = f["Cadastro/salarios"].attrs.get("meses", salarios.shape[0])

# Cria um DataFrame com colunas por pessoa
df = pd.DataFrame(salarios, columns=["Pessoa_1", "Pessoa_2"])

# Soma os salários de todas as pessoas em cada mês (linha)
df["Total_Mensal"] = df.sum(axis=1)

# Adiciona rótulo do mês (1 a 12)
df.index = range(1, meses + 1)
df.index.name = "Mês"

print("📊 Tabela de salários e soma por mês:")
print(df)
print("\nSoma total de todos os meses:", df["Total_Mensal"].sum())


📊 Tabela de salários e soma por mês:
     Pessoa_1  Pessoa_2  Total_Mensal
Mês                                  
1      5000.0    7000.0       12000.0
2      5050.0    7020.0       12070.0
3      5100.0    7050.0       12150.0
4      5200.0    7100.0       12300.0
5      5250.0    7200.0       12450.0
6      5300.0    7250.0       12550.0
7      5400.0    7300.0       12700.0
8      5450.0    7350.0       12800.0
9      5500.0    7400.0       12900.0
10     5550.0    7500.0       13050.0
11     5600.0    7600.0       13200.0
12     5700.0    7700.0       13400.0

Soma total de todos os meses: 151570.0
